<h2>Setting Stuff Up</h2>

Here we import some packages that we'll need in various places.

In [1]:
!mkdir -p ~/agave

%cd ~/agave

!pip3 install --upgrade setvar

import re
import os
import sys
from setvar import *
from time import sleep

# This cell enables inline plotting in the notebook
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

/home/jovyan/agave
Requirement already up-to-date: setvar in /opt/conda/lib/python3.6/site-packages
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


<h2>Set Variables</h2>

The setvar() package gives us a bash-like way of setting environment variables in Python. It's a lot easier to type `PORT=10022` than it is to type `os.environ["PORT"]="10022"`. The setvar package also supports comments and running a command in the background and getting its output.

In [29]:
setvar("""

MACHINE_IP=$VM_IPADDRESS
MACHINE_NAME=jupyter
EMAIL=foo@example.com
MACHINE_USERNAME=jovyan
DOCKERHUB_NAME=dooley

AGAVE_JSON_PARSER=jq
AGAVE_TENANTS_API_BASEURL=https://public.agaveapi.co/tenants

AGAVE_APP_NAME=funwave-tvd-${MACHINE_NAME}-${AGAVE_USERNAME}
AGAVE_APP_DEPLOYMENT_PATH=agave-deployment

AGAVE_SYSTEM_SITE_DOMAIN=agaveplatform.org
AGAVE_SYSTEM_PORT=10022
AGAVE_SYSTEM_HOME_DIR=/home/${MACHINE_USERNAME}
AGAVE_STORAGE_SYSTEM_ID=${MACHINE_NAME}-storage-${AGAVE_USERNAME}

AGAVE_EXECUTION_SYSTEM_ID=${MACHINE_NAME}-exec-${AGAVE_USERNAME}
AGAVE_EXECUTION_SYSTEM_WORK_DIR=/home/${MACHINE_USERNAME}
AGAVE_EXECUTION_SYSTEM_SCRATCH_DIR=/home/${MACHINE_USERNAME}

AGAVE_STORAGE_HOME_DIR=/home/${MACHINE_USERNAME}
REQUESTBIN_URL=$(requestbin-create)
""")

MACHINE_IP=52.15.62.13
MACHINE_NAME=jupyter
EMAIL=foo@example.com
MACHINE_USERNAME=jovyan
DOCKERHUB_NAME=dooley
AGAVE_JSON_PARSER=jq
AGAVE_TENANTS_API_BASEURL=https://public.agaveapi.co/tenants
AGAVE_APP_NAME=funwave-tvd-jupyter-dooley
AGAVE_APP_DEPLOYMENT_PATH=agave-deployment
AGAVE_SYSTEM_SITE_DOMAIN=agaveplatform.org
AGAVE_SYSTEM_PORT=10022
AGAVE_SYSTEM_HOME_DIR=/home/jovyan
AGAVE_STORAGE_SYSTEM_ID=jupyter-storage-dooley
AGAVE_EXECUTION_SYSTEM_ID=jupyter-exec-dooley
AGAVE_EXECUTION_SYSTEM_WORK_DIR=/home/jovyan
AGAVE_EXECUTION_SYSTEM_SCRATCH_DIR=/home/jovyan
AGAVE_STORAGE_HOME_DIR=/home/jovyan
REQUESTBIN_URL=https://requestbin.agaveapi.co/16x9z6w1


If you are running locally using Docker Compose, you will need to pull the ip and port of your reverse tunnel from the sandbox. Uncomment the following command, and enter below

In [30]:
if os.environ.get('USE_TUNNEL') == 'True': 
    # fetch the hostname and port of the reverse tunnel running in the sandbox 
    # so Agave can connect to our local sandbox
    !echo $(ssh -q -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null sandbox 'curl -s  http://localhost:4040/api/tunnels | jq -r '.tunnels[0].public_url'') > ngrok_url.txt  
    !cat ngrok_url.txt | sed 's|^tcp://||'
    !cat ngrok_url.txt | sed 's|^tcp://||' | sed -r 's#(.*):(.*)#\1#' > ngrok_host.txt
    !cat ngrok_url.txt | sed 's|^tcp://||' | sed -r 's#(.*):(.*)#\2#' > ngrok_port.txt

    # set the environment variables otherwise set when running in a training cluster
    os.environ['VM_PORT'] = readfile('ngrok_port.txt').strip()
    os.environ['VM_MACHINE'] = readfile('ngrok_host.txt').strip()
    os.environ['AGAVE_SYSTEM_HOST'] = readfile('ngrok_host.txt').strip()
    os.environ['AGAVE_SYSTEM_PORT'] = readfile('ngrok_port.txt').strip()
    !echo "VM_PORT=$VM_PORT"
    !echo "VM_MACHINE=$VM_MACHINE"
    setvar("VM_IPADDRESS=$(getent hosts ${VM_MACHINE}|cut -d' ' -f1)")

0.tcp.ngrok.io:12873
Reading file `ngrok_port.txt'
Reading file `ngrok_host.txt'
Reading file `ngrok_host.txt'
Reading file `ngrok_port.txt'
VM_PORT=12873
VM_MACHINE=0.tcp.ngrok.io
VM_IPADDRESS=52.15.72.79


<h2>PushBullet</h2>
If you wish to use pushbullet to receive status updates, then run the cell below and enter your pushbullet token.

In [21]:
readpass("PBTOK")

Password or secret: PBTOK
Reading file `PBTOK.txt'


<h2>Agave Password</h2>

We'll be running several commands that require the Agave password. We'll store it here for convenience.

In [22]:
if not os.environ.get('AGAVE_PASSWORD') :
    readpass("AGAVE_PASSWORD")

<h2>Setting up Agave</h2>
Agave uses machines called "tenants" to manage user login and authentication. In this step, we tell Agave we are using the standard proction tenant.

In [23]:
!tenants-init -t agave.prod

You are now configured to interact with the APIs at https://public.agaveapi.co/


<h3>Create the Client</h3>

In this next step we delete the client if it exists. Chances are, yours doesn't yet. We put this command here in case, for some reason, you want to re-create your client later on. If you delete the client you intend to create before you create it, no harm is done.

In [24]:
!clients-delete $AGAVE_APP_NAME

Successfully deleted client funwave-tvd-jupyter-dooley


In this step we create the client. Clients provide a way of encapsulating resources connected to a single project. Through the client, you will receive a token which you can use to run most of the Agave commands.

In [ ]:
!clients-create -S -N $AGAVE_APP_NAME

Create the token for your client. You will, from this point on, use this token to run the remainder of the Agave commands in this tutorial.

In [ ]:
!auth-tokens-create 